In [ ]:
""" #For colab
!pip install \
'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q roboflow supervision
!wget -q \
'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'
"""

: 

In [ ]:
import torch
from segment_anything import sam_model_registry

#DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE = torch.device('cpu')
MODEL_TYPE = "vit_b"

sam = sam_model_registry[MODEL_TYPE](checkpoint="checkpoints/sam_vit_b.pth")
#sam.to(DEVICE)

: 

In [ ]:
import cv2
from matplotlib import pyplot as plt

image_bgr = cv2.imread("ejemplos_fibras/Imagen001_Overlay.jpg")
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
image_rgb = cv2.resize(image_rgb, (128, 128))
plt.imshow(image_rgb)

: 

In [ ]:

from segment_anything import SamAutomaticMaskGenerator

#image_rgb = torch.tensor(image_rgb).permute(2, 0, 1).to(DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)
mask = mask_generator.generate(image_rgb)

: 

In [ ]:
print(mask[0].keys())
print(len(mask))

: 

In [ ]:
plt.imshow(mask[3]['segmentation'].astype(int)*255)

: 

In [ ]:
import numpy as np

def draw_masks_fromDict(image, masks_generated) :
  masked_image = image.copy()
  for i in range(len(masks_generated)) :
    masked_image = np.where(np.repeat(masks_generated[i]['segmentation'].astype(int)[:, :, np.newaxis], 3, axis=2),
                            np.random.choice(range(256), size=3),
                            masked_image)

    masked_image = masked_image.astype(np.uint8)

  return cv2.addWeighted(image, 0.3, masked_image, 0.7, 0)

: 

In [ ]:
segmented_image = draw_masks_fromDict(image_rgb, mask)

plt.imshow(segmented_image, cmap='gray')

: 

In [ ]:
n_mask = len(mask)

for n in n_mask:
    cv2.imwrite("masks/mask_{}.png".format(n), mask[n]['segmentation'].astype(int)*255)



In [ ]:
"""
from PIL import Image
import os

images = []
for file in os.listdir("ejemplos_fibras"):
    if file.endswith(".jpg"):
        images.append(Image.open(os.path.join("ejemplos_fibras", file)))

import random
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i in range(3):
    axs[i].imshow(np.array(images[random.randint(0, len(images))]))
    axs[i].axis("off")
"""

: 